In [12]:
import requests
from bs4 import BeautifulSoup

# First, try to scrape attendance value from specific match URL

In [13]:
urls = [
    "https://www.worldfootball.net/report/eerste-klasse-a-2022-2023-krc-genk-sv-zulte-waregem/",
    "https://www.worldfootball.net/report/league-one-2007-2008-millwall-fc-tranmere-rovers/",
    "https://www.worldfootball.net/report/league-one-2020-2021-ipswich-town-fleetwood-town/",
    "https://www.worldfootball.net/report/first-division-2016-2017-apoel-nikosia-anorthosis-famagusta-fc/"
]

for URL in urls:
    print(f"Fetching data for: {URL}")

    # Fetch the webpage
    response = requests.get(URL)
    response.raise_for_status()  # Will raise an exception if there's an error

    # Parse the content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    results = soup.find(id="site").find_all("td", class_="dunkel")

    attendance_td = None
    for i, td in enumerate(results):
        img = td.find('img', title='Attendance')
        if img:
            attendance_td = results[i + 1]  # Get the next <td> element after the img
            break

    if attendance_td:
        attendance = attendance_td.get_text(strip=True)
        print(f"Attendance: {attendance}")
    else:
        print("Attendance not found.")
    print("--------------------")  # To separate results for clarity


Fetching data for: https://www.worldfootball.net/report/eerste-klasse-a-2022-2023-krc-genk-sv-zulte-waregem/
Attendance: 14.111
--------------------
Fetching data for: https://www.worldfootball.net/report/league-one-2007-2008-millwall-fc-tranmere-rovers/
Attendance: 8.925
--------------------
Fetching data for: https://www.worldfootball.net/report/league-one-2020-2021-ipswich-town-fleetwood-town/
Attendance: without spectators.
--------------------
Fetching data for: https://www.worldfootball.net/report/first-division-2016-2017-apoel-nikosia-anorthosis-famagusta-fc/
Attendance not found.
--------------------


# Next, try to scrape attendance values from all matches in specific game week